In [2]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [10]:
# Discover and catalog available data files
data_dir = Path('raw_data')  # Start with raw_data directory

# Check if extracted subdirectory exists and use it if available
extracted_dir = data_dir / 'extracted'
if extracted_dir.exists():
    data_dir = extracted_dir
    print(f"Using extracted data directory: {data_dir.absolute()}")
else:
    print(f"Using raw data directory: {data_dir.absolute()}")
    
if not data_dir.exists():
    print(f"Directory not found: {data_dir.absolute()}")
    print("Please adjust the data_dir path to match your file location")

# Use a generator to find all CSV files recursively
def find_csv_files(directory):
    """Generator that yields CSV files from a directory recursively"""
    for path in Path(directory).rglob('*.csv'):
        yield path

# Use the generator but store results for display purposes
all_csv_files = list(find_csv_files(data_dir))
print(f"Found {len(all_csv_files)} CSV files in the dataset")

# File size calculation can also use a generator approach
def get_file_sizes(files):
    """Generator that yields file paths with their sizes"""
    for file_path in files:
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        yield file_path, size_mb

# Display ALL files with their sizes
print("\nComplete list of all CSV files:")
for i, (file_path, size_mb) in enumerate(get_file_sizes(all_csv_files)):
    print(f"{i+1}. {file_path} - {size_mb:.2f} MB")


Using extracted data directory: /home/liran/Documents/medicare_data_project/raw_data/extracted
Found 18 CSV files in the dataset

Complete list of all CSV files:
1. raw_data/extracted/beneficiary_2019.csv - 3.92 MB
2. raw_data/extracted/beneficiary_2021.csv - 4.34 MB
3. raw_data/extracted/beneficiary_2020.csv - 4.13 MB
4. raw_data/extracted/beneficiary_2022.csv - 4.56 MB
5. raw_data/extracted/beneficiary_2015.csv - 3.15 MB
6. raw_data/extracted/outpatient.csv - 320.09 MB
7. raw_data/extracted/beneficiary_2017.csv - 3.48 MB
8. raw_data/extracted/beneficiary_2023.csv - 4.84 MB
9. raw_data/extracted/inpatient.csv - 33.89 MB
10. raw_data/extracted/beneficiary_2016.csv - 3.31 MB
11. raw_data/extracted/carrier.csv - 443.66 MB
12. raw_data/extracted/dme.csv - 36.49 MB
13. raw_data/extracted/hospice.csv - 4.38 MB
14. raw_data/extracted/beneficiary_2018.csv - 3.69 MB
15. raw_data/extracted/snf.csv - 9.49 MB
16. raw_data/extracted/beneficiary_2025.csv - 3.17 MB
17. raw_data/extracted/hha.csv - 2

In [14]:
# Cell 3 (Revised): Determine correct file format and create summary
import pandas as pd
import os
from pathlib import Path

# Let's investigate the first few lines of a file to determine the delimiter
sample_file = all_csv_files[0]
with open(sample_file, 'r', errors='ignore') as f:
    first_line = f.readline().strip()
    second_line = f.readline().strip() if f.readline() else ""
    
print(f"Sample from {sample_file.name}:")
print(f"First line: {first_line[:100]}...")
print(f"Second line: {second_line[:100]}...")

# Check potential delimiters
potential_delimiters = [',', '|', '\t', ';']
delimiter_counts = {}

for delimiter in potential_delimiters:
    delimiter_counts[delimiter] = first_line.count(delimiter)
    
print("\nPotential delimiters detected:")
for delimiter, count in delimiter_counts.items():
    print(f"  '{delimiter}': {count} occurrences")
    
# Determine most likely delimiter
most_likely_delimiter = max(delimiter_counts, key=delimiter_counts.get)
print(f"\nMost likely delimiter: '{most_likely_delimiter}' with {delimiter_counts[most_likely_delimiter]} occurrences")

# Try reading with the detected delimiter
print(f"\nAttempting to read files with '{most_likely_delimiter}' delimiter...")

# Process files with correct delimiter
file_info = []
for file_path in all_csv_files:
    try:
        # Read just the first few rows with detected delimiter
        df_sample = pd.read_csv(file_path, delimiter=most_likely_delimiter, nrows=5)
        
        # Get file size in MB
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        
        file_info.append({
            'filename': file_path.name,
            'directory': str(file_path.parent),
            'columns': len(df_sample.columns),
            'size_mb': round(size_mb, 2),
            'column_sample': list(df_sample.columns[:3])
        })
        print(f"Processed {file_path.name} - {size_mb:.2f} MB with {len(df_sample.columns)} columns")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Create a summary dataframe
files_df = pd.DataFrame(file_info)

# Display the file summary sorted by size
print("\nFile Summary (sorted by size):")
display(files_df[['filename', 'columns', 'size_mb']].sort_values('size_mb', ascending=False))

# Simple categorization function
def get_file_category(filename):
    filename = filename.lower()
    if 'beneficiary' in filename:
        return 'Beneficiary'
    elif any(x in filename for x in ['carrier', 'inpatient', 'outpatient', 'dme', 'hha', 'hospice', 'snf']):
        return 'Claims'
    else:
        return 'Other'
        
# Add simple categorization
files_df['category'] = files_df['filename'].apply(get_file_category)

# Group by category
category_summary = files_df.groupby('category').agg({
    'filename': 'count', 
    'size_mb': 'sum'
}).reset_index()

print("\nFiles by Category:")
display(category_summary)

Sample from beneficiary_2019.csv:
First line: BENE_ID|STATE_CODE|COUNTY_CD|ZIP_CD|BENE_BIRTH_DT|SEX_IDENT_CD|BENE_RACE_CD|ENTLMT_RSN_ORIG|ENTLMT_R...
Second line: -10000010254647|01|1410|35756|14-Aug-1990|1|2|1|1|0|0|0||2019|12|12|12|0|0|CME|| |A|29|06-Feb-2014|0...

Potential delimiters detected:
  ',': 0 occurrences
  '|': 184 occurrences
  '	': 0 occurrences
  ';': 0 occurrences

Most likely delimiter: '|' with 184 occurrences

Attempting to read files with '|' delimiter...
Processed beneficiary_2019.csv - 3.92 MB with 185 columns
Processed beneficiary_2021.csv - 4.34 MB with 185 columns
Processed beneficiary_2020.csv - 4.13 MB with 185 columns
Processed beneficiary_2022.csv - 4.56 MB with 185 columns
Processed beneficiary_2015.csv - 3.15 MB with 185 columns
Processed outpatient.csv - 320.09 MB with 162 columns
Processed beneficiary_2017.csv - 3.48 MB with 185 columns
Processed beneficiary_2023.csv - 4.84 MB with 185 columns
Processed inpatient.csv - 33.89 MB with 197 columns
Proces

,filename,columns,size_mb
10,carrier.csv,96,443.66
5,outpatient.csv,162,320.09
11,dme.csv,87,36.49
8,inpatient.csv,197,33.89
14,snf.csv,160,9.49
17,beneficiary_2024.csv,185,5.09
7,beneficiary_2023.csv,185,4.84
3,beneficiary_2022.csv,185,4.56
12,hospice.csv,86,4.38
1,beneficiary_2021.csv,185,4.34



Files by Category:


,category,filename,size_mb
0,Beneficiary,11,43.68
1,Claims,7,850.09


In [17]:
# Cell 4: Basic Data Structure Exploration
print("EXPLORING MEDICARE DATA STRUCTURE")
print("=" * 50)

# Function to analyze file schema
def analyze_schema(file_path, delimiter='|', sample_rows=5):
    """Analyze and print basic schema information for a file"""
    try:
        # Get file size
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        
        # Count total rows (efficiently)
        total_rows = sum(1 for _ in open(file_path, 'r'))
        
        # Read sample to get columns
        df_sample = pd.read_csv(file_path, delimiter=delimiter, nrows=sample_rows)
        
        print(f"\nFile: {file_path.name}")
        print(f"Size: {size_mb:.2f} MB")
        print(f"Total rows: {total_rows:,}")
        print(f"Total columns: {len(df_sample.columns)}")
        
        # Print schema information
        print("\nSchema:")
        schema_info = []
        for col in df_sample.columns:
            # Get data type
            dtype = str(df_sample[col].dtype)
            
            # Get a sample non-null value if available
            sample = str(df_sample[col].dropna().iloc[0]) if not df_sample[col].isna().all() else "NULL"
            if len(sample) > 50:
                sample = sample[:47] + "..."
                
            # Check if all values are the same
            unique_count = df_sample[col].nunique()
            
            schema_info.append({
                'Column': col,
                'Type': dtype,
                'Sample Value': sample,
                'Unique Values': unique_count,
                'Has Nulls': df_sample[col].isna().any()
            })
        
        # Display schema as a table
        schema_df = pd.DataFrame(schema_info)
        display(schema_df)
        
        return total_rows, len(df_sample.columns), df_sample
        
    except Exception as e:
        print(f"Error analyzing {file_path}: {e}")
        return 0, 0, None

# First, get an overview of all files
print("OVERALL DATASET SUMMARY")
print("-" * 50)
file_summary = []

for file_path in all_csv_files:
    try:
        # Get basic info without full schema
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        df_sample = pd.read_csv(file_path, delimiter='|', nrows=1)
        file_summary.append({
            'filename': file_path.name,
            'size_mb': round(size_mb, 2),
            'columns': len(df_sample.columns)
        })
        
    except Exception as e:
        print(f"Error with {file_path.name}: {e}")

# Create summary table
summary_df = pd.DataFrame(file_summary)
print("\nOverall File Summary:")
display(summary_df.sort_values('size_mb', ascending=False))

# Group files by type
def get_file_type(filename):
    filename = filename.lower()
    if 'beneficiary' in filename:
        year = filename.split("_")[-1].split(".")[0] if "_" in filename else "unknown"
        return f"Beneficiary {year}"
    elif 'carrier' in filename:
        return "Carrier Claims"
    elif 'inpatient' in filename:
        return "Inpatient Claims"
    elif 'outpatient' in filename:
        return "Outpatient Claims"
    elif 'dme' in filename:
        return "DME Claims"
    elif 'hha' in filename:
        return "Home Health Claims"
    elif 'hospice' in filename:
        return "Hospice Claims"
    elif 'snf' in filename:
        return "Skilled Nursing Claims"
    else:
        return "Other"

summary_df['file_type'] = summary_df['filename'].apply(get_file_type)
type_summary = summary_df.groupby('file_type').agg({
    'filename': 'count',
    'size_mb': 'sum',
    'columns': 'mean'
}).reset_index()

print("\nFiles by Type:")
display(type_summary.sort_values('size_mb', ascending=False))

# Now, analyze schema for each type of file (one example of each)
print("\n\nDETAILED SCHEMA ANALYSIS")
print("=" * 50)

# Get one example of each file type
file_types = {}
for file_path in all_csv_files:
    file_type = get_file_type(file_path.name)
    if file_type not in file_types:
        file_types[file_type] = file_path

# Analyze schema for each file type
dataframes = {}
for file_type, file_path in file_types.items():
    print(f"\n{'=' * 30}")
    print(f"ANALYZING {file_type.upper()}")
    print(f"{'=' * 30}")
    
    rows, cols, df_sample = analyze_schema(file_path, delimiter='|', sample_rows=10)
    
    # Store sample dataframe for later use
    if df_sample is not None:
        dataframes[file_type] = df_sample
        
        # For the largest files, show just the column names to avoid clutter
        if file_path.name in ['carrier.csv', 'outpatient.csv']:
            print("\nColumns (first 20):")
            for i, col in enumerate(df_sample.columns[:20]):
                print(f"{i+1}. {col}")
            if len(df_sample.columns) > 20:
                print(f"...and {len(df_sample.columns) - 20} more columns")

# Explore potential primary keys and foreign keys
print("\n\nPOTENTIAL JOIN KEYS")
print("=" * 50)

# Look for common ID columns across files
all_id_columns = {}
for file_type, df in dataframes.items():
    # Identify potential ID columns
    id_cols = [col for col in df.columns if 'ID' in col.upper() and 'PROVIDER' not in col.upper()]
    all_id_columns[file_type] = id_cols
    
    print(f"\n{file_type} potential ID columns:")
    for col in id_cols[:5]:
        print(f"  - {col}")
    if len(id_cols) > 5:
        print(f"  - ...and {len(id_cols) - 5} more")

# Look for common ID columns that might be join keys
print("\nPotential join keys between files:")
for file_type1, id_cols1 in all_id_columns.items():
    for file_type2, id_cols2 in all_id_columns.items():
        if file_type1 != file_type2:
            common_cols = set(id_cols1).intersection(set(id_cols2))
            if common_cols:
                print(f"{file_type1} <-> {file_type2}: {', '.join(common_cols)}")

EXPLORING MEDICARE DATA STRUCTURE
OVERALL DATASET SUMMARY
--------------------------------------------------

Overall File Summary:


,filename,size_mb,columns
10,carrier.csv,443.66,96
5,outpatient.csv,320.09,162
11,dme.csv,36.49,87
8,inpatient.csv,33.89,197
14,snf.csv,9.49,160
17,beneficiary_2024.csv,5.09,185
7,beneficiary_2023.csv,4.84,185
3,beneficiary_2022.csv,4.56,185
12,hospice.csv,4.38,86
1,beneficiary_2021.csv,4.34,185



Files by Type:


,file_type,filename,size_mb,columns
11,Carrier Claims,1,443.66,96.0
16,Outpatient Claims,1,320.09,162.0
12,DME Claims,1,36.49,87.0
15,Inpatient Claims,1,33.89,197.0
17,Skilled Nursing Claims,1,9.49,160.0
9,Beneficiary 2024,1,5.09,185.0
8,Beneficiary 2023,1,4.84,185.0
7,Beneficiary 2022,1,4.56,185.0
14,Hospice Claims,1,4.38,86.0
6,Beneficiary 2021,1,4.34,185.0




DETAILED SCHEMA ANALYSIS

ANALYZING BENEFICIARY 2019

File: beneficiary_2019.csv
Size: 3.92 MB
Total rows: 7,447
Total columns: 185

Schema:


...


ANALYZING HOME HEALTH CLAIMS

File: hha.csv
Size: 2.09 MB
Total rows: 6,216
Total columns: 88

Schema:


,Column,Type,Sample Value,Unique Values,Has Nulls
0,BENE_ID,int64,-10000010254850,1,False
1,CLM_ID,int64,-10000930044285,1,False
2,NCH_NEAR_LINE_REC_IDENT_CD,object,W,1,False
3,NCH_CLM_TYPE_CD,int64,10,1,False
4,CLM_FROM_DT,object,14-Oct-2021,1,False
5,CLM_THRU_DT,object,09-Nov-2021,1,False
6,NCH_WKLY_PROC_DT,object,12-Nov-2021,1,False
7,FI_CLM_PROC_DT,float64,NULL,0,True
8,PRVDR_NUM,int64,17879,1,False
9,CLM_FAC_TYPE_CD,int64,3,1,False



ANALYZING BENEFICIARY 2024

File: beneficiary_2024.csv
Size: 5.09 MB
Total rows: 9,661
Total columns: 185

Schema:


,Column,Type,Sample Value,Unique Values,Has Nulls
0,BENE_ID,int64,-10000010254618,10,False
1,STATE_CODE,int64,1,1,False
2,COUNTY_CD,int64,1500,9,False
3,ZIP_CD,int64,36109,10,False
4,BENE_BIRTH_DT,object,16-Aug-1999,10,False
...,...,...,...,...,...
180,CST_SHR_GRP_CD_08,int64,2,4,False
181,CST_SHR_GRP_CD_09,int64,2,4,False
182,CST_SHR_GRP_CD_10,int64,2,4,False
183,CST_SHR_GRP_CD_11,int64,2,4,False




POTENTIAL JOIN KEYS

Beneficiary 2019 potential ID columns:
  - BENE_ID
  - SEX_IDENT_CD
  - VALID_DEATH_DT_SW
  - PTC_CNTRCT_ID_01
  - PTC_CNTRCT_ID_02
  - ...and 58 more

Beneficiary 2021 potential ID columns:
  - BENE_ID
  - SEX_IDENT_CD
  - VALID_DEATH_DT_SW
  - PTC_CNTRCT_ID_01
  - PTC_CNTRCT_ID_02
  - ...and 58 more

Beneficiary 2020 potential ID columns:
  - BENE_ID
  - SEX_IDENT_CD
  - VALID_DEATH_DT_SW
  - PTC_CNTRCT_ID_01
  - PTC_CNTRCT_ID_02
  - ...and 58 more

Beneficiary 2022 potential ID columns:
  - BENE_ID
  - SEX_IDENT_CD
  - VALID_DEATH_DT_SW
  - PTC_CNTRCT_ID_01
  - PTC_CNTRCT_ID_02
  - ...and 58 more

Beneficiary 2015 potential ID columns:
  - BENE_ID
  - SEX_IDENT_CD
  - VALID_DEATH_DT_SW
  - PTC_CNTRCT_ID_01
  - PTC_CNTRCT_ID_02
  - ...and 58 more

Outpatient Claims potential ID columns:
  - BENE_ID
  - CLM_ID
  - NCH_NEAR_LINE_REC_IDENT_CD
  - REV_CNTR_IDE_NDC_UPC_NUM

Beneficiary 2017 potential ID columns:
  - BENE_ID
  - SEX_IDENT_CD
  - VALID_DEATH_DT_SW
  -